import needed library and declare list of category column

In [145]:

import csv
import pandas as pd
import random
import nltk
from nltk.tokenize import wordpunct_tokenize
from collections import defaultdict
from nltk.probability import FreqDist
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sklearn.metrics import classification_report

crawledDataSetNames = ['dataset-reddit-business.csv', 'dataset-reddit-entertainment.csv', 
'dataset-reddit-parenting.csv', 'dataset-reddit-politics.csv', 
'dataset-reddit-sports.csv', 'dataset-reddit-travel.csv']

categories = ['business', 'entertainment', 'parenting', 'politics', 'sports', 'travel']




preprocessing function

In [146]:
def preprocessing(originData):
    data = originData.copy()
    #make all characters to lower case
    data['text']=data['text'].apply(lambda x: str.lower(x))

    #word tokenization
    data['text'] = data['text'].apply(lambda x: wordpunct_tokenize(x))

    #delete non alphabets word
    data['text'] = data['text'].apply(lambda x: list(word for word in x if word.isalpha()))

    print('preprocessing finished. 1 / 6')
    return data

declare making word dictionary function and making Bag Of words Column function.

In [147]:
def makingWordDict(originData):
    data = originData.copy()
    listedText=data['text'].tolist()
    allTextList = []
    for l in listedText:
        allTextList.extend(l)
    numOfdict = len(set(allTextList))
    
    numOfFeatureWords = int(numOfdict*1)

    fdist = FreqDist(allTextList)

    wordDict = list(word for word, freq in fdist.most_common(numOfFeatureWords))

    print('making word dictionary finished. 2 / 6')
    return wordDict

def makingBagOfWordsCol(originData, wordDict):
    data = originData.copy()
    def bagOfWords(tokens):
        d = defaultdict(int,{ word:0 for word in wordDict })

        for token in tokens:
            d[token]+=1

        return list(d.values())[:len(wordDict)]

    data['bagOfWords'] = data['text'].apply(lambda x: bagOfWords(x))

    print('making bag of words column finished. 3 / 6')
    return data

declare divide trainset and testset function.

In [148]:
def divideTrainAndTest(originData):
    data = originData.copy()
    dataSize = data.shape[0]

    trainSplit = 0.7
    testSplit = 0.3

    trainSize = int(dataSize * trainSplit)
    testSize = int(dataSize * testSplit)

    print('trainSize: ', trainSize)
    print('testSize: ', testSize)

    trainSet = data.iloc[:trainSize, :]
    testSet = data.iloc[trainSize:, :]

    print('dividing train and test set finished. 4 / 6')
    return trainSet, testSet

declare build a Bag of Words model and fit by using train set.

In [149]:
def buildAndFitModel(trainSet):
    classifier = MultinomialNB()
    trainX = np.array(trainSet['bagOfWords'].tolist())
    trainY = np.array(trainSet['category'].tolist())
    
    classifier.fit(trainX, trainY)

    print('building and fitting the model finished. 5 / 6')
    return classifier

declare evaluate function.

In [150]:
def evaluate(classifier, testSet):
    testX = np.array(testSet['bagOfWords'].tolist())
    testY = np.array(testSet['category'].tolist())
    
    predY = classifier.predict(testX)
    predYProb = classifier.predict_proba(testX)
    accuracy = accuracy_score(testY, predY)
    f1Score = 

    print('evaluating the model finished. 6 / 6')
    return accuracy, predYProb

1. read the crawled Dataset of reddit.

In [151]:
rawRedditDataset = pd.DataFrame(columns = {'text','category'})
for crawledDataSetName in crawledDataSetNames:
    #read csv files
    df = pd.read_csv('./dataset/%s'%crawledDataSetName)
    
    #replace Nan to empty string
    df = df.fillna('')

    #'titleAndContent' Column is concat of 'title' and 'content'
    df['text']=df['title']+' '+df['content']
    rawRedditDataset = pd.concat([rawRedditDataset, df[['text', 'category']]])

rawRedditDataset=rawRedditDataset[['text', 'category']]
#shuffle row
rawRedditDataset=rawRedditDataset.sample(frac=1).reset_index(drop=True)

print(rawRedditDataset)

                                                   text       category
0     Chinese Embassy voices 'grave concern' over ac...         sports
1     I don't regret being a parent My 2 year old so...      parenting
2     Miss USA 2019 Cheslie Kryst Dead at 30 from Su...  entertainment
3                 Please Stop Trying To Sell Me Crypto   entertainment
4     My son thinks he can swim but he cannot Hi the...      parenting
...                                                 ...            ...
5936  [TooGooden] Players currently in Covid protoco...         sports
5937                  My 2020 trip to the Amalfi Coast          travel
5938  Sean Penn In Ukraine: Putin Has Made a 'Horrib...  entertainment
5939  McCarthy threatens to strip GOP members of com...       politics
5940          My first time in magical Petra (Jordan).          travel

[5941 rows x 2 columns]


2. read the news category dataset(from kaggle).

In [152]:
originNewsCategory = ['BUSINESS', 'ENTERTAINMENT', 'PARENTING', 'POLITICS', 'SPORTS', 'TRAVEL']

rawNewsDataset = pd.read_json("./dataset/News_category_Dataset_v2.json", lines=True)
rawNewsDataset = rawNewsDataset[rawNewsDataset['category'].isin(originNewsCategory)]
rawNewsDataset = rawNewsDataset.groupby('category').sample(n=2000)
rawNewsDataset['category'] = rawNewsDataset['category'].apply(lambda x: x.lower())

rawNewsDataset['text'] = rawNewsDataset['headline'] + ' ' + rawNewsDataset['short_description']
rawNewsDataset = rawNewsDataset[['text','category']]

#shuffle row
rawNewsDataset=rawNewsDataset.sample(frac=1).reset_index(drop=True)

print(rawNewsDataset)

                                                    text       category
0      Luke Bryan And Karen Fairchild Get The Crowd R...  entertainment
1      Miley Cyrus Isn't Sorry About That Controversi...  entertainment
2      Ed O’Neill Once Mistook Britney Spears For A B...  entertainment
3      There's No Silver Bullet For Solving School Lu...       politics
4      Dirty I cannot brush my hair today/There is no...      parenting
...                                                  ...            ...
11995  'How Many Reds Does It Take To Catch A Pop Fly...         sports
11996  Obama Wants His Alaska Trip To Be The 'Punctua...       politics
11997  Entrepreneur Leadership Today Demands a Human ...       business
11998  Don't Say This: The Cruelest, Most Ignorant Pa...      parenting
11999  Marissa Mayer Calls Out Media For Sexist Cover...       business

[12000 rows x 2 columns]


3. the model only using reddit crawled dataset.

In [153]:
redditDataset = preprocessing(rawRedditDataset)
redditWordDict = makingWordDict(redditDataset)
BOWredditDataset = makingBagOfWordsCol(redditDataset, redditWordDict)

redditTrainSet, redditTestSet = divideTrainAndTest(BOWredditDataset)

redditClassifier = buildAndFitModel(redditTrainSet)

redditAccuracy, redditPredYProb = evaluate(redditClassifier, redditTestSet)

print('only reddit dataset model accuracy: ', redditAccuracy)

preprocessing finished. 1 / 6
making word dictionary finished. 2 / 6
making bag of words column finished. 3 / 6
trainSize:  4158
testSize:  1782
dividing train and test set finished. 4 / 6
building and fitting the model finished. 5 / 6
evaluating the model finished. 6 / 6
only reddit dataset model accuracy:  0.8457655636567583


4. the model only using news dataset.

In [154]:
newsDataset = preprocessing(rawNewsDataset)
newsWordDict = makingWordDict(newsDataset)
BOWnewsDataset = makingBagOfWordsCol(newsDataset, newsWordDict)

newsTrainSet, newsTestSet = divideTrainAndTest(BOWnewsDataset)

newsClassifier = buildAndFitModel(newsTrainSet)

newsAccuracy, newsPredYProb = evaluate(newsClassifier, newsTestSet)

print('only news dataset model accuracy: ', newsAccuracy)

preprocessing finished. 1 / 6
making word dictionary finished. 2 / 6
making bag of words column finished. 3 / 6
trainSize:  8400
testSize:  3600
dividing train and test set finished. 4 / 6
building and fitting the model finished. 5 / 6
evaluating the model finished. 6 / 6
only news dataset model accuracy:  0.7830555555555555


5. the model using both reddit and news datasets.

In [155]:
rawBothDataset = pd.concat([rawRedditDataset, rawNewsDataset])

bothDataset = preprocessing(rawBothDataset)
bothWordDict = makingWordDict(bothDataset)
BOWbothDataset = makingBagOfWordsCol(bothDataset, bothWordDict)

bothTrainSet, bothTestSet = divideTrainAndTest(BOWbothDataset)

bothClassifier = buildAndFitModel(bothTrainSet)

bothAccuracy, bothPredYProb = evaluate(bothClassifier, bothTestSet)

print('using both reddit and both dataset model accuracy: ', bothAccuracy)

preprocessing finished. 1 / 6
making word dictionary finished. 2 / 6
making bag of words column finished. 3 / 6
trainSize:  12558
testSize:  5382
dividing train and test set finished. 4 / 6
building and fitting the model finished. 5 / 6
evaluating the model finished. 6 / 6
using both reddit and both dataset model accuracy:  0.7505108675459781


6. predict category of reddit testset by the model only trained by news train set.

In [156]:
BOWredditTestsetUsingNewsWordDict = makingBagOfWordsCol(redditTestSet, newsWordDict)

redditOnNewsAccuracy, redditOnNewsPredYProb = evaluate(newsClassifier, BOWredditTestsetUsingNewsWordDict)

print('predicting the category of reddit test set on news-trained model accuracy: ', redditOnNewsAccuracy)


/var/folders/4h/gkd9rvld5sd12215ppr2x1pc0000gn/T/ipykernel_7770/1202364807.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['bagOfWords'] = data['text'].apply(lambda x: bagOfWords(x))


making bag of words column finished. 3 / 6
evaluating the model finished. 6 / 6
predicting the category of reddit test set on news-trained model accuracy:  0.802019068984857


7. predict category of news testset by the model only trained by reddit train set.

In [157]:
BOWnewsTestsetUsingRedditWordDict = makingBagOfWordsCol(newsTestSet, redditWordDict)

newsOnRedditAccuracy, newsOnRedditPredYProb = evaluate(redditClassifier, BOWnewsTestsetUsingRedditWordDict)

print('predicting the category of reddit test set on news-trained model accuracy: ', newsOnRedditAccuracy)


/var/folders/4h/gkd9rvld5sd12215ppr2x1pc0000gn/T/ipykernel_7770/1202364807.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['bagOfWords'] = data['text'].apply(lambda x: bagOfWords(x))


making bag of words column finished. 3 / 6
evaluating the model finished. 6 / 6
predicting the category of reddit test set on news-trained model accuracy:  0.5166666666666667
